In [ ]:
import torch
torch.set_default_dtype(torch.float64)

In [ ]:
from collections import deque

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import tdg

To take an infinite volume limit of the free theory ($\tilde{a}=0$) requires thinking.  Recall that $ \tilde{\beta} = \frac{\beta}{ML^2}$ and we want to adjust L at constant $\beta$.

The issue is that we have no dimensionful way to fix L.  However, we can *scale* L in the following way.

Fix a particular starting L0 (which you don't have to specify), nx0 and $\tilde{\beta}_0$.  Then as we change nx, if we kept the lattice spacing the same we would need to scale L by (nx/nx0); $L = (nx/nx0) L_0$.  Therefore $\tilde{\beta} = \tilde{\beta}_0 (nx0 / nx)^2$.

We can make a similar scaling for $\tilde{\mu} = \mu ML^2 $ which should go to $\tilde{\mu} = \tilde{\mu}_0 (nx/nx0)^2$.  But, we'll just set $\mu=0$ and then scaling 0 leaves it fixed.  The same thing happens for $\tilde{h}$.

In [ ]:
nx0 = 7
beta0 = 3./49
mu0 = 0.
h0 = torch.tensor([0.,0.,0.])

In [ ]:
NX = (7, 9, 11, 13, 15, 17, 19, 21, 23, 25,)

In [ ]:
ensembles = deque()

In [ ]:
for nx in NX:
    L = tdg.Lattice(nx)
    spacetime = tdg.Spacetime(2, L)
    
    beta = beta0 * (nx0 / nx)**2
    mu   = mu0   * (nx / nx0)**2
    h    = h0    * (nx / nx0)**2
    
    S = tdg.Action(
        spacetime,
        tdg.Potential(-0.000000000000000001*tdg.LegoSphere([0,0])),
        beta=beta,
        mu=mu,
        h=h,
        )
    
    zero_configuration = spacetime.vector()
    cfgs = torch.stack((zero_configuration,))
    
    free = tdg.ensemble.GrandCanonical(S).from_configurations(cfgs).bootstrapped(1)
    ensembles.append(free)

In [ ]:
fig, ax = plt.subplots(11,1, sharex='col', figsize=(8,30))

for e in ensembles:
    
    L = e.Action.Spacetime.Lattice
    nx = L.nx
    k2 = L.linearize(L.ksq)

    ax[0].plot(((nx0/nx)**2,), (e.N[0].real/nx**2,        ), label=f'{nx}', marker='o')
    ax[1].plot(((nx0/nx)**2,), (e.T_by_TF[0].real,  ), label=f'{nx}', marker='o')

    ax[2].plot(((nx0/nx)**2,), (e.kinetic_by_kF4[0].real,        ), label=f'{nx}', marker='o')
    ax[3].plot(((nx0/nx)**2,), (e.internalEnergy_by_kF4[0].real,  ), label=f'{nx}', marker='o')
    
    ax[4].plot(((nx0/nx)**2,), (e.n_momentum[0,0].real,  ), label=f'{nx}', marker='o')
    
    ax[-6].plot(((nx0/nx)**2,), (e.w0_by_kF4[0].real, ), label=f'{nx}', marker='o')
    ax[-5].plot(((nx0/nx)**2,), (e.w2_by_kF2[0].real, ), label=f'{nx}', marker='o')
    ax[-4].plot(((nx0/nx)**2,), (e.w4[0].real,        ), label=f'{nx}', marker='o')
    
    ax[-3].plot(((nx0/nx)**2,), (e.b2_by_kF4[0].real,), label=f'{nx}', marker='o')
    ax[-2].plot(((nx0/nx)**2,), (e.b4_by_kF2[0].real,), label=f'{nx}', marker='o')
    ax[-1].plot(((nx0/nx)**2,), (e.b6       [0].real,), label=f'{nx}', marker='o')
    
ax[0].set_ylabel('N / nx^2')
ax[1].set_ylabel('T/TF')

ax[2].set_ylabel('k / kF^4')
ax[3].set_ylabel('u / kF^4')
ax[4].set_ylabel('n(k=0) / L^2')

ax[-6].set_ylabel('w0 / kF^4')
ax[-5].set_ylabel('w2 / kF^2')
ax[-4].set_ylabel('w4')

ax[-3].set_ylabel('b2 / kF^4')
ax[-2].set_ylabel('b4 / kF^2')
ax[-1].set_ylabel('b6')


ax[-1].set_xlim([0, 1.05])
ax[-1].set_xlabel('(nx0 / nx)^2')

# Momentum-dependent quantities

To study the momentum-dependence as we go towards the infinite volume limit, we should hold the physical momentum absolutely fixed for a fair comparison.

Since integer lattice momenta don't correspond to fixed physical momenta $\vec{k} = 2\pi \vec{n}/L$ which depends on $L$.  We could think: fix $L_0$ and show things as a function of $(k L_0 / 2\pi)^2 = \vec{n} (nx_0 / nx)^2$.  This succeeds.

But, perhaps more physically, we can show $(k/k_F)^2$.  Since $k_F$ is intensive it doesn't scale as we take the infinite volume limit.

In [ ]:
fig, ax = plt.subplots(2,1, sharex='col', figsize=(8,12))

for e in ensembles:
    
    L = e.Action.Spacetime.Lattice
    nx = L.nx
    nt = e.Action.Spacetime.nt
    k2 = L.linearize(L.ksq)

    ax[0].plot(e.momentum_by_kF_squared[0].real, e.n_momentum[0].real,    label=f'{nx=}', marker='o', linestyle='none', zorder=-nx)
    ax[1].plot(e.momentum_by_kF_squared[0].real, e.spin_momentum[0].real, label=f'{nx=}', marker='o', linestyle='none', zorder=-nx)

ax[0].set_yscale('log')
ax[1].set_yscale('symlog')
    
ax[0].set_ylabel('n(k) / L^2')
ax[1].set_ylabel('s(k) / L^2')

ax[0].set_xlabel('(k/kF)^2')
ax[0].legend(ncol=2)

# Condensate Fraction

In the free theory all the condensate fractions should be zero; a plot does not show anything more informative than a bunch of zeros.

In [ ]:
cutoff = 1e-10

for e in ensembles:
    print(
        e.Action.Spacetime.Lattice.nx,
        (e.pairing_singlet[0].abs() < cutoff).unique(return_counts=True),
        e.condensate_fraction_singlet[0],
        e.condensate_fraction_triplet_plus[0],
        e.condensate_fraction_triplet_zero[0],
        e.condensate_fraction_triplet_minus[0],
        e.condensate_fraction_up_down[0],
    )

In [ ]:
fig, AX = plt.subplots(4,5, sharex='col', sharey='row', figsize=(20,15))

for e in ensembles:
    
    L = e.Action.Spacetime.Lattice
    nx = L.nx
    k2 = L.linearize(L.ksq)

    for ax, EIGENVALUES in zip(AX, (
        e.pair_pair_eigenvalues_singlet, 
        e.pair_pair_eigenvalues_triplet_plus,
        e.pair_pair_eigenvalues_triplet_zero,
        e.pair_pair_eigenvalues_triplet_minus,
    )):
    
        for a, obs in zip(ax, (EIGENVALUES[0].flip(dims=(0,)))):# e.pair_pair_triplet_plus, e.pair_pair_triplet_zero, e.pair_pair_triplet_minus, e.pair_pair_up_down)):

            a.plot(((nx0/nx)**2,), (obs, ), label=f'{nx}', marker='o')

for (a, label) in zip(AX[:,0], ('singlet', 'triplet plus', 'triplet zero', 'triplet minus')):
    a.set_ylabel(label)

for (a, label) in zip(AX[0], ('largest eigenvalue', 'next largest', '...', 'and so on', '...')):
    a.set_title(label)

            
ax[-1].set_xlim([0, 1.05])
for a in ax:
    a.set_xlabel('(nx0 / nx)^2')
